In [1]:
!pip install soynlp

     |████████████████████████████████| 416 kB 4.2 MB/s 


In [2]:
!git clone https://github.com/grayfactory/ReviwReactionBot.git
%cd ReviwReactionBot/

Cloning into 'ReviwReactionBot'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 6), reused 23 (delta 4), pack-reused 0
Unpacking objects: 100% (25/25), done.
/content/ReviwReactionBot


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
# simbolic link
!ln -s /content/gdrive/My\ Drive/chat_bot/ReviewReactionBot /chat_bot
!ls /chat_bot

checkpoints	preprocessing.ipynb  Seq2Seq_model.py
data		preprocessing.py     Seq2Seq_Transformer.ipynb
df_save.pkl	__pycache__	     small_transformer.png
lightning_logs	README.md	     soynlp_word_extractor.pkl
model_chp	requirements.txt     train_koGPT2.py
preprocessing	ReviewReactionBot    visualize.ipynb


In [30]:
from pathlib import Path
import pandas as pd
import time
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import numpy as np

# custom functions
from Seq2Seq_model import transformer, CustomSchedule, loss_function
from preprocessing import pre_processing_text

In [31]:
class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value

def get_data_csv_path(configs, condition):
  condition = condition
  return [f for f in configs.data_list if f'{condition}' in str(f)][0]

configs = MyConfig({
    'path_drive': Path('/chat_bot'), 
    'vocab_size': 32000, # hyperparameter same as koGTP2
    'MAX_LENGTH': 128 # hyperparameter same as koGTP2
})

configs.data_list = [f for f in (configs.path_drive / 'data').glob('*')]

In [32]:
df = pd.read_csv(get_data_csv_path(configs,'ChatbotData_v2.csv'))
df = df.dropna()
df.head()

,Unnamed: 0,Q,A,star_avg
0,12,와 진짜 대박 맛있 어서 암청 많이 먹다가 리뷰 하는거 까먹어서 지금 남기네요 ㄷㄷ...,맛있 게 드셨다니 저희 도 정말 기쁩니다 ㅎㅎ 더 맛있 고 정성 스럽게 조리 할테니...,5.0
1,13,치킨 이 양이 적어 보이는데 치킨 덩어리 가 엄청 커요!! 목덕후인 저는 목이 없어...,소중한 사진 리뷰 감사 합니다 ! 앞으로 도 더 맛있 는 음식 으로 찾아 뵐게요 ~...,4.0
2,14,처음 시켜 보는데 상담히 만족 합니다. 배달 주문 하기 힘드신 어머님 이 부탁 하셔...,첫 주문 에 만족 하셨다니 정말 다행이에요 ㅎㅎ 앞으로 도 초심 잃지 않고 저희 모...,5.0
3,15,맛있 게 잘 먹었습니다,소중한 리뷰 감사 합니다 ! 더 맛있 고 정성 스러운 음식 으로 찾아 뵐게요 좋은 ...,5.0
4,16,맛있 었어요. 감사 합니다.,맛있 게 드셨다니 정말 다행이에요 감사 합니다 ! 좋은 하루 보내 시고 항상 건강 ...,5.0


# 1. Tokenized with tensorflow SubwordTextEncoder

In [33]:
text_list = [text for text in df.Q.to_list() + df.A.to_list() if text] # if None False
len(text_list)

1145822

In [34]:
%%time
# tf 서브워드텍스트인코더를 사용하여 [질문, 답변] 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_list, target_vocab_size=configs.vocab_size)

# 시작 토큰과 종료 토큰에 대한 정수 부여(vocav size의 마지막 두 숫자). 
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

CPU times: user 7min 22s, sys: 1.23 s, total: 7min 23s
Wall time: 7min 23s


In [35]:
# tokenizer 작동 확인
print(f'Vocab size:{tokenizer.vocab_size}')
# 임으의 문장 encodign test
print(tokenizer.encode(text_list[20]))
print(tokenizer.encode('이 집 짬뽕 맛있네요.'))
for ts in tokenizer.encode('이 집 짬뽕 맛있네요.'):
  print(f'{ts} -- {tokenizer.decode([ts])}')

Vocab size:32190
[284, 271, 794, 2551, 306, 8, 2118, 21, 48, 79, 223, 1524, 599, 2808, 284, 2118, 164, 58, 11, 18, 572, 25349]
[10, 665, 3801, 4252, 31980]
10 -- 이 
665 -- 집 
3801 -- 짬뽕 
4252 -- 맛있네요
31980 -- .


In [36]:
# train data, sentence pair Tokenizing

def tokenize_and_filter(inputs, outputs):
  tokenize_inputs, tokenize_outputs = [], []

  for (sent1, sent2) in zip(inputs, outputs):

    sent1 = START_TOKEN + tokenizer.encode(sent1) + END_TOKEN
    sent2 = START_TOKEN + tokenizer.encode(sent2) + END_TOKEN

    tokenize_inputs.append(sent1)
    tokenize_outputs.append(sent2)

  # padding
  tokenize_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenize_inputs, maxlen=configs.MAX_LENGTH, padding='post'
  )
  tokenize_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenize_outputs, maxlen=configs.MAX_LENGTH, padding='post'
  )

  return tokenize_inputs, tokenize_outputs

review_encode, answer_encode = tokenize_and_filter( df['Q'], df['A'] )

# 최종 학습 데이터 pair
print(review_encode.shape, answer_encode.shape)

(572911, 128) (572911, 128)


# 2. Training Transformer model
### Data batch 구성

In [37]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
configs.BATCH_SIZE = 128 
configs.BUFFER_SIZE = 20000 # 버퍼에 올릴 데이터 size & shuffle

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': review_encode,
        'dec_inputs': answer_encode[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {   # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. # 교사 강요을 위한 output(정답)
        'outputs': answer_encode[:, 1:] 
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(configs.BUFFER_SIZE)
dataset = dataset.batch(configs.BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [38]:
# Hyper-parameters
configs.D_MODEL = 256
configs.NUM_LAYERS = 2
configs.NUM_HEADS = 8
configs.DFF = 512
configs.DROPOUT = 0.1
configs.VOCAB_SIZE = VOCAB_SIZE
configs.EPOCHS = 5

configs.learning_rate = CustomSchedule(configs.D_MODEL)

configs.optimizer = tf.keras.optimizers.Adam(
    configs.learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [39]:
configs

{'BATCH_SIZE': 128,
 'BUFFER_SIZE': 20000,
 'DFF': 512,
 'DROPOUT': 0.1,
 'D_MODEL': 256,
 'EPOCHS': 5,
 'MAX_LENGTH': 128,
 'NUM_HEADS': 8,
 'NUM_LAYERS': 2,
 'VOCAB_SIZE': 32192,
 'data_list': [PosixPath('/chat_bot/data/df_강남구_세곡동.pkl'),
  PosixPath('/chat_bot/data/df_강남구_개포동.pkl'),
  PosixPath('/chat_bot/data/df_강남구_자곡동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_송파1동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_송파2동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_석촌동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_문정1동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_문정2동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_오륜동.pkl'),
  PosixPath('/chat_bot/data/df_송파구_오금동.pkl'),
  PosixPath('/chat_bot/data/df_종로구_사직동.pkl'),
  PosixPath('/chat_bot/data/df_종로구_삼청동.pkl'),
  PosixPath('/chat_bot/data/df_종로구_부암동.pkl'),
  PosixPath('/chat_bot/data/df_강남구_청담동.pkl'),
  PosixPath('/chat_bot/d

In [40]:
# check-points
configs.checkpoint_path = str(configs.path_drive / 'checkpoints' / 'cp-{epoch:04d}.ckpt')
# "/content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

In [41]:
tf.keras.backend.clear_session()

model = transformer(
    vocab_size=configs.VOCAB_SIZE,
    num_layers=configs.NUM_LAYERS,
    dff=configs.DFF,
    d_model=configs.D_MODEL,
    num_heads=configs.NUM_HEADS,
    dropout=configs.DROPOUT)


def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, configs.MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=configs.optimizer, loss=loss_function, metrics=[accuracy])


# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=configs.checkpoint_path,
    save_weights_only=True,
    verbose=1
    # period= # 만약 1 epoch씩 저장하지 않으면
    )

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(configs.checkpoint_path.format(epoch=0))

(1, 32192, 256)
(1, 32192, 256)


In [ ]:
model.fit(dataset, 
          epochs=configs.EPOCHS,
          callbacks=[cp_callback])

# 최종 모델 Checkpoint load

In [ ]:
# 최종 checkpoint load
latest = tf.train.latest_checkpoint('/chat_bot/checkpoints/')
print(latest)

# 새로운 model 객체
model = transformer(
    vocab_size=configs.VOCAB_SIZE,
    num_layers=configs.NUM_LAYERS,
    dff=configs.DFF,
    d_model=configs.D_MODEL,
    num_heads=configs.NUM_HEADS,
    dropout=configs.DROPOUT)

model.compile(optimizer=configs.optimizer, loss=loss_function, metrics=[accuracy])
model.load_weights(latest)

/chat_bot/checkpoints/cp-0010.ckpt
(1, 31974, 256)
(1, 31974, 256)


In [ ]:
def evaluate(sentence):
  sentence = pre_processing_text(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(configs.MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

output = predict('이 집 짬뽕 진짜 맛있네요!')

Input: 이 집 짬뽕 진짜 맛있네요!
Output: 안녕하세요 논현 스쿨푸드입니다. 배달도 빠르고 맛있다니 최고의 칭찬입니다. 항상 노력하는 논현 스쿨푸드를 사랑해주세요.


In [ ]:
new_sentence = ['이 집 짱뽕 진짜 맛있네요',
                '음식이 다 식어서 왔어여,,, ㅠㅠ',
                '공짜로 줘도 다시는 안먹는다ㅎㅎㅎㅎㅎ',
                '공짜로 줘도 다시는 안먹는다.',
                'ㄷㄷㄷㄷ']

for text in new_sentence:
  predict(text)
  print('\n')

Input: 이 집 짱뽕 진짜 맛있네요
Output: 자주 이용해 주시고 소중한 리뷰 주셔서 감사드립니다. 항상 맛있게 드셔주신다는 말씀에 앞으로도 변함없이 행복한 맛을 느끼시도록 최선을 다하여 준비하겠습니다. 웃음 넘치는 하루 보내시길 바랍니다.


Input: 음식이 다 식어서 왔어여,,, ㅠㅠ
Output: 사랑하는 고객님, 저희 음식을 주문해주셔서 감사합니다 배달이 지연되어 죄송합니다 ㅠㅠ 맛있는 음식 빠르게 받으실수있도록 노력하겠습니다. 오늘하루도 건강하고 행복한하루되세요~ （╹◡╹）


Input: 공짜로 줘도 다시는 안먹는다ㅎㅎㅎㅎㅎ
Output: 맛있게 드셔주셔서 정말 감사합니다~~!! 저희 기본 소스 자체에 매콤한맛이 조금 조금 조금 덜 맵게 해달라고 요청사항에 적어주시면 잘 준비해드리겠습니다!!


Input: 공짜로 줘도 다시는 안먹는다.
Output: 맛있게 드셔주셔서 정말 감사합니다~~!! 저희 기본 소스 자체에 매콤한맛이 조금 조금 조금 덜 맵게 해달라고 요청사항에 적어주시면 잘 준비해드리겠습니다!!


Input: ㄷㄷㄷㄷ
Output: 소중한 리뷰 감사드립니다♡ 언제나 감동을 드릴 수 있는 맛과 서비스로 보답해드리겠습니다! 앞으로도 저희 명도찜닭 많은 이용 부탁드립니다 건강 조심하시고, 다음에 또 주문주세요~!


